<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import re

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
 def create_df(host, title, author, n_pages, publishing, language, binding, isbn):
  cols = ['Loja', 'Título', 'Autor', 'Nº Páginas', 'Editora', 'Idioma', 'Acabamento', 'ISBN']

  list_info = [host, title, author, n_pages, publishing, language, binding, isbn]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)
  
  return display(book_pd)


## Sites Livros

1.   https://www.amazon.com.br/
2.   https://www.submarino.com.br/
3.   https://www.saraiva.com.br/
4.   https://www.travessa.com.br/
5.   https://www.americanas.com.br/
6.   https://www.ciadoslivros.com.br/
7.   https://www.traca.com.br/
8.   https://www.livrariacultura.com.br
9.   https://www.disal.com.br/
10.  https://www.extra.com.br/livros/

## 1. AMAZON

### All books in one page

In [0]:
def Amazon_col(site):
  page = rq.get(url)

  soup = bs(page.text,"html.parser")

  books = soup.find_all('li', {'class':'a-carousel-card acswidget-carousel__card'})

  books_list = []

  for book in books:
    book = re.sub("\n|\t|\xa0|[R][$]", " ", (book.text.strip()))
    book = re.sub("[,]", '.', book)
    books_list.append(book)
    
  for i in range(len(books_list)):
    books_list[i] = books_list[i].split('    ')
    books_list[i] = list(filter(None, books_list[i]))
    books_list[i] = [item.strip() for item in books_list[i]]
    if (len(books_list[i]) > 4):
      books_list[i] = [books_list[i][0],books_list[i][1],books_list[i][2],books_list[i][4]]

  book_list = []
  
  for book in books_list:
    if len(book) == 4:
      book_list.append(book)

  col = ['Título', 'Autor', 'Acabamento', 'Preço']

  books_pd = pd.DataFrame(book_list,\
                        columns=col)
  
  return display(books_pd.head())

In [0]:
url_list = ['https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3a1_w?node=7872689011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=QVZ9EYBNWVBAX5VRE6KH&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2a1_w?node=16495864011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=G9RQPBBW9G6ZQ5XX82M8&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2d1_w?node=11443295011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=0AC9WCXZN61JJGX49F4H&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4b1_w?node=17021500011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=0JX9SAGKQ4QCZ7DET92M&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2c1_w?node=7882627011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2b1_w?node=16271670011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3b1_w?node=7842710011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3d1_w?node=7872552011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011',
            'https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4d1_w?node=5475882011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011']

In [0]:
for url in url_list:
  print(url)
  Amazon_col(url)
  print('\n')

https://www.amazon.com.br/Livros/b?ie=UTF8&node=6740748011&ref_=nav_books_all


,Título,Autor,Acabamento,Preço
0,Do Mil ao Milhão. Sem Cortar o...,Thiago Nigro,Capa comum,34.90
1,O milagre da manhã,Hal Elrod. Marcelo Schild,Capa comum,39.90
2,A Sutil Arte de Ligar o F*da-Se: Uma...,Mark Manson. Joana Faro,Capa comum,34.90
3,Mais esperto que o Diabo: O mistério...,Napoleon Hill,Capa comum,42.90
4,Os segredos da mente milionária,T. Harv Eker. Pedro Jorgensen Junior,Capa comum,34.90




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3a1_w?node=7872689011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=QVZ9EYBNWVBAX5VRE6KH&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_2a1_w?node=16495864011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=G9RQPBBW9G6ZQ5XX82M8&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


...

,Título,Autor,Acabamento,Preço




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_3d1_w?node=7872552011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço
0,Confeitaria escalafobética...,Raiza Costa,Capa dura,98.00
1,A química dos bolos: Receitas e...,Joyce Galvão,Capa dura,129.90
2,Le Cordon Bleu : Todas as técnicas...,Jeni Wright. Eric Treuille,Capa dura,249.90
3,O Que Tem na Geladeira? 30 Legumes e...,Rita Lobo,Capa dura,86.00
4,Cozinha prática,Rita Lobo,Capa dura,90.00




https://www.amazon.com.br/b/ref=s9_acss_bw_cg_livros_4d1_w?node=5475882011&pf_rd_m=A3RN7G7QC5MWSZ&pf_rd_s=merchandised-search-13&pf_rd_r=068FWS2KD5K5K0RGNXRY&pf_rd_t=101&pf_rd_p=ef6255a0-96c8-4931-bc85-1e4d682f1064&pf_rd_i=6740748011


,Título,Autor,Acabamento,Preço


### Information from a single book

In [0]:
ama_down_list = [ r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/1808.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo Monteiro Lobato Amazon.com.br.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Cotoco. O Diário De Um Garoto De 13 Anos - Livros na Amazon Brasil- 9788598078854.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Do Mil ao Milhão. Sem Cortar o Cafezinho. - 9788595083271 - Livros na Amazon Brasil.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Doctor Who, Neil Gaiman e vários - Livros na Amazon.com.br.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Freud (1900) A interpretação dos sonhos Obras completas volume 4 - 9788535932218 - Livros na Amazon Brasil.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Good Omens Belas Maldições - Livros na Amazon Brasil- 9788528624021.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Os Mitos de Cthulhu - Volume Único (Exclusivo Amazon) Livros.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Redes de Computadores e a Internet Uma Abordagem Top-Down - 9788581436777 - Livros na Amazon Brasil.html',
                  r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Sobotta - Atlas de Anatomia Humana - 3 Volumes - 9788527732376 - Livros na Amazon Brasil.html']



In [0]:
def amazon_single(site):

  soup = bs(open(site),"html.parser")

  title = (soup.find('span', {'id':'productTitle'})).text
  
  author = (soup.find('span', {'class':'author notFaded'})).find('a',{'class':'a-link-normal'}).text
  
  book = soup.find('div', {'class':'content'}).find_all('li')[:5]
 
  n_pages = book[0].contents[1].strip().split(' ')[0]
  
  publishing = book[1].contents[1].strip().split(';')[0]
  
  language = book[2].contents[1].strip()
  
  cols = ['Título', 'Autor', 'Nº Páginas', 'Editora', 'Idioma']

  list_info = [title, author, n_pages, publishing, language]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)
  
  return display(book_pd)

In [0]:
for url in ama_down_list:
  print(url)
  amazon_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/1808.html


,Título,Autor,Nº Páginas,Editora,Idioma
0,"1808: Como uma rainha\n louca, um príncipe med...",Laurentino Gomes,384,Globo Livros,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo Monteiro Lobato Amazon.com.br.html


,Título,Autor,Nº Páginas,Editora,Idioma
0,Box - As Melhores Aventuras Do Sítio Do Picapa...,Monteiro Lobato,640,Nova Fronteira,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Cotoco. O Diário De Um Garoto De 13 Anos - Livros na Amazon Brasil- 9788598078854.html


...

,Título,Autor,Nº Páginas,Editora,Idioma
0,Os Mitos de Cthulhu - Volume Único (Exclusivo ...,Esteban Maroto,92,Pipoca e Nanquim (7 de junho de 2019),Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Redes de Computadores e a Internet Uma Abordagem Top-Down - 9788581436777 - Livros na Amazon Brasil.html


,Título,Autor,Nº Páginas,Editora,Idioma
0,Redes de Computadores e a Internet: Uma Aborda...,Jim Kurose,656,Pearson Universidades,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Sobotta - Atlas de Anatomia Humana - 3 Volumes - 9788527732376 - Livros na Amazon Brasil.html


,Título,Autor,Nº Páginas,Editora,Idioma
0,Sobotta - Atlas de Anatomia Humana - 3 Volumes,Friedrich Paulsen,1168,Guanabara Koogan,Português


## 2. SARAIVA

In [0]:
def saraiva_single(site):
  soup = bs(open(site),"html.parser")

  title = soup.find('div', {'class':'product-review'}).h1.text

  infos = soup.find('table', {'class':'group Caracteristicas'})

  author = infos.find('td', {'class':'value-field Autor'}).text

  n_pages = infos.find('td', {'class':'value-field Numero-de-Paginas'}).text

  language = infos.find('td', {'class':'value-field Idioma'}).text

  binding = infos.find('td', {'class':'value-field Acabamento'}).text

  cols = ['Título', 'Autor', 'Nº Páginas', 'Acabamento', 'Idioma']

  list_info = [title, author, n_pages, binding, language]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)

  return display(book_pd)

In [0]:
sar_url_list = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Alice No País Das Maravilhas (Classic Edition) - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Steve Jobs - A Biografia - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Vade Mecum Compacto - Brochura - 21a Ed. 2019 - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/It - A Coisa - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/O Guia do Mochileiro Das Galáxias - Série o Mochileiro Das Galáxias - Vol. 1 - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/O Livrinho do Bebê - Animais, Palavras, Contrários, Cores e Formas - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Star Wars - O Arquivo Rebelde - Saraiva.html']
# r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Como Fazer Amigos e Influenciar Pessoas - Saraiva.html'
# r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Notebook Gamer Samsung Odyssey Intel Core i5-7300HQ,GEFORCE GTX 1050 4Gb,Hd 1Tb,8Gb,15,W10 - Saraiva.html',


In [0]:
for url in sar_url_list:
  print(url)
  saraiva_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Alice No País Das Maravilhas (Classic Edition) - Saraiva.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Alice No País Das Maravilhas (Classic Edition),"Carroll,Lewis",208,Capa dura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Steve Jobs - A Biografia - Saraiva.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Steve Jobs - A Biografia,"Isaacson,Walter, Isaacson,Walter",624,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Vade Mecum Compacto - Brochura - 21a Ed. 2019 - Saraiva.html


...

,Título,Autor,Nº Páginas,Acabamento,Idioma
0,O Guia do Mochileiro Das Galáxias - Série o Mo...,"Adams,Douglas",208,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/O Livrinho do Bebê - Animais, Palavras, Contrários, Cores e Formas - Saraiva.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,"O Livrinho do Bebê - Animais, Palavras, Contrá...","Books,Yoyo",128,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Star Wars - O Arquivo Rebelde - Saraiva.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Star Wars - O Arquivo Rebelde,"Wallace,Daniel",176,Brochura,Português


##3. LIVRARIA DA TRAVESSA

In [0]:
def travessa_single(site):
  soup = bs(open(site),"html.parser")
  
  infos = soup.find('div', {'id':'divDados'})

  author = soup.find('span', {'id':'lblNomAutor'}).a.text

  title = infos.find('span', {'id':'lblDadosNome'}).text

  n_pages = infos.find('span', {'id':'lblDadosPaginas'}).text

  binding = infos.find('span', {'id':'lblDadosEncadernacao'}).text

  language = infos.find('span', {'id':'lblDadosIdioma'}).text

  cols = ['Título', 'Autor', 'Nº Páginas', 'Acabamento', 'Idioma']

  list_info = [title, author, n_pages, binding, language]
  list_info
  book_pd = pd.DataFrame([list_info],\
                          columns=cols)
  display(book_pd)

In [0]:
trav_url_one = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/A COR DA MAGIA - Terry Pratchett - Livro.html',
                
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/CYNDI MINHA HISTORIA - Cyndi Lauper Jancee Dunn - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/D. PEDRO II A HISTORIA NAO CONTADA - Paulo Rezzutti - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/JAPONES PARA LEIGOS - Eriko Sato - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/LONELY PLANET ARGENTINA (INCLUI URUGUAI) - Globo Livros - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/MADELINE FINN E OS CÃES DO ABRIGO - Lisa Papp - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/O PODER DA AUTORRESPONSABILIDADE A FERRAMENTA COMPROVADA QUE GERA ALTA PERFORMANCE E RESULTADOS EM POUCO TEMPO - Paulo Vieira - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/SO PARA UM ALIMENTAÇAO SAUDAVEL PARA QUEM MORA SOZINHO - Rita Lobo - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/VIDAS SECAS - 140aED.(2019) - Graciliano Ramos - Livro.html']

                # r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/CRONICAS PARA LER EM QUALQUER LUGAR - Vários (ver informações no detalhe) - Livro.html',

In [0]:
for url in trav_url_one:
  print(url)
  travessa_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/A COR DA MAGIA - Terry Pratchett - Livro.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,A COR DA MAGIA,Terry Pratchett,232,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/CYNDI MINHA HISTORIA - Cyndi Lauper Jancee Dunn - Livro.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,CYNDI: MINHA HISTORIA,Cyndi Lauper,352,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/D. PEDRO II A HISTORIA NAO CONTADA - Paulo Rezzutti - Livro.html


...

,Título,Autor,Nº Páginas,Acabamento,Idioma
0,O PODER DA AUTORRESPONSABILIDADE: A FERRAMENTA...,Paulo Vieira,160,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/SO PARA UM ALIMENTAÇAO SAUDAVEL PARA QUEM MORA SOZINHO - Rita Lobo - Livro.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,SO PARA UM: ALIMENTAÇAO SAUDAVEL PARA QUEM MOR...,Rita Lobo,168,Capa dura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/VIDAS SECAS - 140aED.(2019) - Graciliano Ramos - Livro.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,VIDAS SECAS - 140ªED.(2019),Graciliano Ramos,176,Brochura,Português


##4. TRAÇA LIVRARIA

In [0]:
def traca_single(site):
  soup = bs(open(site),"html.parser")

  title = soup.find('div', {'class':'box-principal-dados'}).h1.text

  author = soup.find('div', {'class':'box-principal-dados'}).a.text

  infos = soup.find('ul', {'class':'ul-detalhes'})

  list_info = [info.text.replace('\n','').split(':') for info in infos if info != '\n']

  for info in (list_info):
    if ('Encadernação' in info):
      binding = info[1]
      break

  for info in (list_info):
    if ('Páginas' in info):
      n_pages = info[1]
      break

  language = soup.find('div', {'style':'font-size:18px;line-height:20px;'}).text
  language = re.sub('\n|[L][i][v][r][o][ ][e][m] | (.*?)\s', '', language)

  cols = ['Título', 'Autor', 'Nº Páginas', 'Acabamento', 'Idioma']

  list_info = [title, author, n_pages, binding, language]
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)

  display(book_pd)

In [0]:
traca_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/A Casa Dos Budas Ditosos - João Ubaldo Ribeiro - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cavaleiros Do Zodíaco Vol 1 - Masami Kurumada - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cem Anos De Solidão - Gabriel Garcia Marquez - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Crianças Famosas  Schubert - Ann Rachlin E Susan Hellard - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Indiozinhos - Mônica Haibara - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Livro Da Esperança - Francisco Cândido Xavier - Emmanuel - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Mozart Don Giovanni - A Vida De Mozart - Lorenzo Da Ponte - Henry Stendhal - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/O Colapso Do Universo - Isaac Asimov - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Políticas Territoriais Na Amazônia - Neli Aparecida De Mello - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Relações Exteriores Do Brasil Contemporâneo - Danielly Silva Ramos Becard - Traça Livraria e Sebo.html']

In [0]:
for url in traca_urls:
  print(url)
  traca_single(url)
  print('\n')


/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/A Casa Dos Budas Ditosos - João Ubaldo Ribeiro - Traça Livraria e Sebo.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,A Casa Dos Budas Ditosos,João Ubaldo Ribeiro,164,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cavaleiros Do Zodíaco Vol 1 - Masami Kurumada - Traça Livraria e Sebo.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Cavaleiros Do Zodíaco Vol 1,Masami Kurumada,110,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cem Anos De Solidão - Gabriel Garcia Marquez - Traça Livraria e Sebo.html


...

,Título,Autor,Nº Páginas,Acabamento,Idioma
0,O Colapso Do Universo,Isaac Asimov,210,Capa Dura,em(Brasil)




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Políticas Territoriais Na Amazônia - Neli Aparecida De Mello - Traça Livraria e Sebo.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Políticas Territoriais Na Amazônia,Neli Aparecida De Mello,412,Brochura,Português




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Relações Exteriores Do Brasil Contemporâneo - Danielly Silva Ramos Becard - Traça Livraria e Sebo.html


,Título,Autor,Nº Páginas,Acabamento,Idioma
0,Relações Exteriores Do Brasil Contemporâneo,Danielly Silva Ramos Becard,224,Brochura,Português


## 5. CIA DOS LIVROS

In [0]:
def cia_dos_livros_single(site):
  soup = bs(open(site), 'html.parser')

  title = soup.find('div', {'class':'detailProduct'}).h1.text

  author = soup.find('dl', {'class':'autor-editora'}).dd.contents[0].strip()

  infos = soup.find('dl', {'id':'bookData'})
  infos_list = [info for info in infos]
  infos_list = [re.sub('\n|<.*?>|:|\t', '', str(info)) for info in infos_list]
  infos_list = [item for item in infos_list if item != '']
  
  language = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Idioma'][0]

  n_pages = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Número de páginas'][0]

  binding = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Encadernação'][0]

  publishing = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Marca'][0]

  cols = ['Título', 'Autor', 'Nº Páginas', 'Editora', 'Idioma', 'Acabamento']

  list_info = [title, author, n_pages, publishing, language, binding]
  list_info
  book_pd = pd.DataFrame([list_info],\
                        columns=cols)
  display(book_pd)


In [0]:
cia_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Afirmar-se com Nietzsche - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Audácia Dessa Mulher, A - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Avaliação - Mito e Desafio - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Breve História do Feminismo no Contexto Euro-americano, Uma - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Corpo Fala, O - A Linguagem Silenciosa da Comunicação Não Verbal - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Cura do Ciúme, A - Aprenda a Confiar, Supere a Possessividade e Salve Seu Relacionamento - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Dom Casmurro - Coleção Grandes Mestres da Literatura Brasileira - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Em Busca do Texto Perfeito - Questões Contemporâneas de Edição, Preparação e Revisão Textual - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Manual do Arquiteto Descalço - Capa Dura - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Feminismo É Feminino?, O - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Gramática de Espanhol Para Brasileiros - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Nem Só De Ciência Se Faz A Cura - O Que Os Pacientes Me Ensinaram - Cia. dos Livros.html']

In [0]:
for url in cia_urls:
  print(url)
  cia_dos_livros_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Afirmar-se com Nietzsche - Cia. dos Livros.html


,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento
0,Afirmar-se com Nietzsche,Balthasar Thomass,208,Vozes Nobilis,Português,Brochura




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Audácia Dessa Mulher, A - Cia. dos Livros.html


,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento
0,"Audácia Dessa Mulher, A",Ana Maria Machado,408,Alfaguara,Português,Brochura




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Avaliação - Mito e Desafio - Cia. dos Livros.html


...

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento
0,"Feminismo É Feminino?, O",Maíra Marcondes Moreira,204,Annablume,Português,Brochura




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Gramática de Espanhol Para Brasileiros - Cia. dos Livros.html


,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento
0,Gramática de Espanhol Para Brasileiros,Esther Maria Milani,432,Saraiva - Didaticos,Português,Brochura




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Nem Só De Ciência Se Faz A Cura - O Que Os Pacientes Me Ensinaram - Cia. dos Livros.html


,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento
0,Nem Só De Ciência Se Faz A Cura: O Que Os Paci...,Protasio L. Da Luz,304,Manole,Português,Brochura


##6. EXTRA

In [0]:
def extra_single(site):
  soup = bs(open(site), 'html.parser')
  
  title = soup.find('div', {'class':'produtoNome'}).h1.b.text

  author = soup.find('dl', {'class':'Autor'}).contents
  author = [re.sub('\n|<.*?>', '', str(item)) for item in author]
  author = [item.strip() for item in author if item != '']
  author = [author[i+1] for i in range(len(author)) if author[i] == 'Autor'][0]

  infos = soup.find_all('div', {'class':'wrp'})
  infos = [(re.sub('\n|<.*?>|\\n', '', str(item))).split() for item in infos][1]

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma'][0]

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas'][0]

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora'][0]

  binding = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Acabamento'][0]

  isbn = [infos[i+1] for i in range(len(infos)) if (infos[i] == 'ISBN' or infos[i] == 'ISBN-13')][0]
 
  host = 'Extra'

  create_df(host, title, author, n_pages, publishing, language, binding, isbn)

In [0]:
extra_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Aromaterapia e as Emoções - Como Usar Óleos Essenciais Para Equilibrar o Corpo e a Mente - Especialidades Médicas no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Once Upon A Time - Irmãos Grimm - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Os Miseráveis - Walcyr Carrasco - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Panelinha - Receitas Que Funcionam - Rita Lobo - Culinária no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - A Pequena Filosofia da Mafalda - Vamos à Sopa - Joaquim Salvador Lavado (Quino) - Infantis no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - A Seleção_ 35 Garotas e uma Coroa - Volume 1 - Kiera Cass - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Como Eu Era Antes de Você - Jojo Moyes - Romance no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Elas Podem... e Devem - o Livro que vai Mexer com Você - Flaviane Brandemberg - Sexologia no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Em Busca de Sentido - Um Psicólogo no Campo de Concentração - Psicologia no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Eu, Fernando Pessoa - em Quadrinhos - Susana Ventura - Quadrinhos Adultos no Extra.com.br.html']

In [0]:
frames = []
for url in extra_urls:
  print(url)
  frames.append(extra_single(url))
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Aromaterapia e as Emoções - Como Usar Óleos Essenciais Para Equilibrar o Corpo e a Mente - Especialidades Médicas no Extra.com.br.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Extra,Aromaterapia e as Emoções: Como Usar Óleos Ess...,Shirley Price,322,Bertrand,Português,Brochura,9788528608519




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Once Upon A Time - Irmãos Grimm - Juvenil no Extra.com.br.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Extra,Livro - Once Upon A Time - Irmãos Grimm,Irmãos Grimm,272,Planeta,Português,Brochura,9788542203080




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Os Miseráveis - Walcyr Carrasco - Juvenil no Extra.com.br.html


...

,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Extra,Livro - Elas Podem... e Devem: o Livro que vai...,Flaviane Brandemberg,114,Ser,Português,Brochura,8563178393




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Em Busca de Sentido - Um Psicólogo no Campo de Concentração - Psicologia no Extra.com.br.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Extra,Livro - Em Busca de Sentido: Um Psicólogo no C...,Viktor E. Frankl,186,Vozes,Português,Brochura,8532606261




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Eu, Fernando Pessoa - em Quadrinhos - Susana Ventura - Quadrinhos Adultos no Extra.com.br.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Extra,"Livro - Eu, Fernando Pessoa: em Quadrinhos - S...",Susana Ventura,72,Peirópolis,Português,Brochura,9788575963050


##7. Cultura

In [0]:
def cultura_single(site):
  soup = bs(open(site),"html.parser")

  title = soup.find('h1', {'class':'title'}).text

  author = soup.find('ul', {'class':'info'}).a.text

  infos = soup.find('ul', {'id':'product-list-detail'}).find_all('li')
  infos = [(re.sub('\n|<.*?>|\\n|:', '', str(item))).split() for item in infos] 

  isbn = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'ISBN'][0]

  language = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'Idioma'][0]

  n_pages = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Páginas'][0]

  publishing = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Editora'][0]

  binding = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Encadernação'][0]

  host = 'Livraria Cultura'

  create_df(host, title, author, n_pages, publishing, language, binding, isbn)

In [0]:
cultura_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - MAIS ESPERTO QUE O DIABO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O CAIBALION - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O PODER DO HÁBITO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - QUEM PENSA ENRIQUECE - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - SOCIEDADE DO CANSAÇO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro GESTALT-TERAPIA - FUNDAMENTOS EPISTEMOLOGICOS | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro INTRODUÇAO A ALGEBRA LINEAR COM APLICAÇOES | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O DIÁRIO DE ANNE FRANK | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O MUNDO SEGUNDO FELIPE NETO | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O PEQUENO PRINCIPE (LIVRO DE BOLSO) | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro PRÓLOGO, ATO, EPÍLOGO | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro QUEM PENSA ENRIQUECE | Livraria Cultura.html']

In [0]:
for url in cultura_urls:
  print(url)
  cultura_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - MAIS ESPERTO QUE O DIABO - Livraria Cultura.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Livraria Cultura,MAIS ESPERTO QUE O DIABO,"HILL, NAPOLEON",200,CDG,PORTUGUÊS,BROCHURA,8568014003




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O CAIBALION - Livraria Cultura.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Livraria Cultura,O CAIBALION,TRES INICIADOS,128,PENSAMENTO,PORTUGUÊS,BROCHURA,8531500710




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O PODER DO HÁBITO - Livraria Cultura.html


...

,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Livraria Cultura,O PEQUENO PRINCIPE (LIVRO DE BOLSO),"SAINT-EXUPERY, ANTOINE DE",96,CAMINHO,PORTUGUÊS,BROCHURA,8589987337




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro PRÓLOGO, ATO, EPÍLOGO | Livraria Cultura.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Livraria Cultura,"PRÓLOGO, ATO, EPÍLOGO","MONTENEGRO, FERNANDA",328,COMPANHIA,PORTUGUÊS,BROCHURA,8535932550




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro QUEM PENSA ENRIQUECE | Livraria Cultura.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Livraria Cultura,QUEM PENSA ENRIQUECE,"HILL, NAPOLEON",248,FUNDAMENTO,PORTUGUÊS,BROCHURA,8576766302


## 8. DISAL

In [0]:
def disal_single(site):
  soup = bs(open(site), 'html.parser')

  info_container = soup.find('div', {'class':'book-title'}).contents
  info_container = [re.sub('\n|<.*?>', '', str(info)) for info in info_container]
  info_container = [info.strip() for info in info_container if info != '']

  author = info_container[1]

  title = info_container[0]

  publishing = info_container[2]

  infos = soup.find('table', {'class':'table table-striped'}).find_all('tr')
  infos = [(re.sub('\n|<.*?>|\\n', '', str(item))).split(':') for item in infos]

  isbn = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'ISBN' or info[i] == 'ISBN 13'][0]

  language = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'Idioma' or info[i] == 'Idioma '][0]

  n_pages = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Páginas' or info[i] == 'Número de páginas'][0]

  binding = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Encadernação' or info[i] == 'Tipo de produto '][0]

  host = 'Disal'

  create_df(host, title, author, n_pages, publishing, language, binding, isbn)

In [0]:
disal_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Ambiente E Os Processos De Maturacao, O - 9788573074567.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Autorretrato E Outras Crônicas - 9788501112330.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Aventuras Na Netoland Com Luccas Neto - 9788555461002.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Exames Laboratoriais E Diagnosticos Em Enfermagem - 9o Ed - 9788527726634.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Fisiologia Endocrina (lange) - 4o Ed - 9788580553918.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Gestao Hospitalar - Para Uma Administracao Eficaz - 9788527733298.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Gramatica Da Lingua Portuguesa Para Leigos - 9788550803395.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Introducao A Analise Matematica - 9788521201687.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Alemao - Alemao-portugues - Portugues-alemao - 9788506078563.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Espanhol - Espanhol-portugues _ Portugues-espanhol - 9788506078532.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Frances - Frances-portugues _ Portugues-frances - 3a Ed - 9788506078549.html']

In [0]:
for url in disal_urls:
  print(url)
  disal_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Ambiente E Os Processos De Maturacao, O - 9788573074567.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Disal,"Ambiente E Os Processos De Maturacao, O","Winnicott, Dona",268,Artmed,Português,Livro,9788573074567




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Autorretrato E Outras Crônicas - 9788501112330.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Disal,Autorretrato E Outras Crônicas,"Andrade, Drummond De Andrade",256,Record,Português,Livro,9788501112330




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Aventuras Na Netoland Com Luccas Neto - 9788555461002.html


...

,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Disal,Michaelis Minidicionario Alemao - Alemao-portu...,Melhoramentos,480,Melhoramentos,Alemão-português / português-alemão,Livro,9788506078563




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Espanhol - Espanhol-portugues _ Portugues-espanhol - 9788506078532.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Disal,Michaelis Minidicionario Espanhol - Espanhol-p...,Melhoramentos,462,Melhoramentos,Português - Espanhol,Livro,9788506078532




/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Frances - Frances-portugues _ Portugues-frances - 3a Ed - 9788506078549.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Disal,Michaelis Minidicionario Frances - Frances-por...,Melhoramentos,510,Melhoramentos,Português - Francês,Livro,9788506078549


##9. Americanas

In [0]:
def americanas_single(site):
  soup = bs(open(site), 'html.parser')

  infos = soup.find_all('table')
  infos = [info.find_all('td') for info in infos]
  infos = [info.text for info in infos[0]]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13'][0]

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma '][0]

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas'][0]

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato'][0]

  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Título'][0]

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor'][0]

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora'][0]

  host = 'Americanas'

  create_df(host, title, author, n_pages, publishing, language, binding, isbn)

In [0]:
americanas_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Cinco Passos de Você nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Corrente nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - As 13 Maldições - Edição Econômica nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Crônicas Saxônicas: A Guerra do Lobo nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Eu Sou Eric Zimmerman nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Herdeiro de Sevenwaters - Coleção Sevenwaters - Vol. 4 nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Historia Da Filosofia Crista nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Medicina dos Horrores nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - O Cadete e o Capitão nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Paixão Libertadora - Desejo Proibido - Vol. 2 nas Lojas Americanas.com.html']

In [133]:

for url in americanas_urls:
  print(url)
  americanas_single(url)
  print('\n')


/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Cinco Passos de Você nas Lojas Americanas.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Americanas,A cinco passos de você,Rachael Lippincott,288,Globo Alt,Português,Livro brochura,9788525067425




/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Corrente nas Lojas Americanas.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Americanas,A corrente,Adrian McKinty,378,Record,Português,Livro brochura,9788501117656




/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - As 13 Maldições - Edição Econômica nas Lojas Americanas.com.html


...

,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Americanas,Medicina Dos Horrores: A história de Joseph Li...,Lindsey Fizharris,320,Intrínseca,Português,Livro Capa Dura,9788551005224




/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - O Cadete e o Capitão nas Lojas Americanas.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Americanas,O cadete e o capitão,Luiz Maklouf Carvalho,256,Todavia,Português,Livro brochura,9786580309351




/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Paixão Libertadora - Desejo Proibido - Vol. 2 nas Lojas Americanas.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Americanas,Paixão libertadora,Sophie Jackson,352,Editora Arqueiro,Português,Livro brochura,9788580415421


## 10. Submarino

In [0]:
def submarino_single(site):
  soup = bs(open(site), 'html.parser')

  infos = soup.find_all('table')
  infos = [info.find_all('td') for info in infos]
  infos = [info.text for info in infos[0]]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13'][0]

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma '][0]

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas'][0]

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato'][0]

  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Título'][0]

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor'][0]

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora'][0]

  host = 'Submarino'

  create_df(host, title, author, n_pages, publishing, language, binding, isbn)

In [0]:
submarino_urls = [
r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/LIvro - Luccas Neto em Os Aventureiros no Submarino.com.html',
r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - A Dama Mais Apaixonada no Submarino.com.html',
r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - O Labirinto do Fauno no Submarino.com.html',
r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Sandman - Edição Definitiva no Submarino.com.html',
r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Seja Foda! no Submarino.com.html']

In [151]:
for url in submarino_urls:
  print(url)
  submarino_single(url)
  print('\n')

/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/LIvro - Luccas Neto em Os Aventureiros no Submarino.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Submarino,Luccas Neto em os aventureiros,Luccas Neto,64,Pixel,Português,Livro brochura,9788555461927




/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - A Dama Mais Apaixonada no Submarino.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Submarino,A dama mais apaixonada,"Julia Quinn, Eloisa James, Connie Brockway",288,Editora Arqueiro,Português,Livro brochura,9788530600211




/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - O Labirinto do Fauno no Submarino.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Submarino,O Labirinto Do Fauno,"Guillermo del Toro, Cornelia Funke",320,Intrínseca,Português,Livro Capa Dura,9788551005194




/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Sandman - Edição Definitiva no Submarino.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Submarino,ABSOLUTE SANDMAN VOL. 1,Neil Gaiman,88,Panini Books,Português,Livro Capa Dura,9788573516531




/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Seja Foda! no Submarino.com.html


,Loja,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Submarino,Seja Foda! - Black Edition,Caio Carneiro,208,Buzz Editora,Português,Livro Capa Dura,9786580435173
